# Over the years, which countries have been the focus of parliamentary discussions?

In this tutorial we will look at votes that focus on events in, or relations with, countries. We won't be concerned with the outcomes of these votes, but focus on their frequency over time - seeing how current, geopolitical topics are reflected in Parliament.

To follow allong with the tutorial, you should already be familiar with data analysis in Python using `pandas`. You don’t need prior knowledge about the European Parliament.


## Context: The European Union Vocabulary

The institutions of the European Union maintain [common vocabularies](https://op.europa.eu/en/web/eu-vocabularies) for many areas. One of this is the [Corporate list of countries and territories](https://op.europa.eu/en/web/eu-vocabularies/countries-and-territories).
This vocabulary provides a list of countries acknowledged by the European Union as well as agreed upon terminology for other relevant territories.

Because the European Parliament provides relevant terms from this vocabulary for all applicable votes, we can heavily rely on this for our task at hand!

## Retrieving the Votes

We begin by using the `votes` table from the HowTheyVote.eu data set. This will give us a list of all roll-call votes in the Parliaments plenary.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
votes_df = pd.read_csv('data/votes.csv')
votes_df.head()

For the contents of this tutorial, we are only concerned with the overall topic of texts (also called reports in the Parliament). Therefore, we can safely filter all votes on amendments from the `votes` table. For more information on amendments, take a look at [one of our other tutorials](https://howtheyvote.github.io/tutorials/lab/index.html?path=close_amendment_votes.ipynb).

In [ ]:
reports = votes_df[votes_df["is_main"] == True].copy()
reports.head()

This essentially leaves us with a list of all final texts that parliament held a roll-call vote on.

## Finding Countries related to Votes

Now, one approach to identify with which countries each text was concerned could be to take a look at the `procedure_title` column and to try to find all countries in there. Extracting parts of unstructured text is always error prone and there are additional hurdles to this approach, for example we would need to bring our own list of countries and regions.

Luckily, by relying on the *Countries and territories vocabulary* mentioned above, the HowTheyVote.eu data set provides us with a significantly better approach by utilizing the `geo_area_votes` table. It shows us the `geo_area_code`s for each vote that relates to specific geopgrahic areas:

In [ ]:
geo_areas_df = pd.read_csv('data/geo_area_votes.csv')
geo_areas_df.head()

As of course multiple texts can be concerned with the same countries, the entries in the `geo_area_code`column are not unique.
We can also quickly confirm that the entries in the `vote_id` column are not unqiue as well:

In [ ]:
vote_counts = geo_areas_df['vote_id'].value_counts().reset_index()
vote_counts.head(5)

Therefore, we can have many reports all concerned with the same country and we can also have reports that are concerned with more than one country.

We match each report with all it's relevant geo codes. This will lead to reports getting duplicated it they relate to more than one geo code, but allows us to easily tally the codes afterwards:

In [ ]:
reports_with_areas = reports.merge(geo_areas_df, left_on="id", right_on="vote_id")
reports_with_areas

Now, we could for example easily construct a list of the top 5 most mentioned countries:

In [ ]:
top_countries = reports_with_areas['geo_area_code'].value_counts().head(5).reset_index()
top_countries

To make this list easier to understand, we can also retrieve the full names of the geo areas. These are stored in the `geo_areas.csv` and can easily be joined onto our table, which contains the `code` of the areas:

In [ ]:
geo_areas = pd.read_csv("data/geo_areas.csv")
geo_areas.head()

In [ ]:
top_countries.merge(geo_areas[["code", "label"]], left_on="geo_area_code", right_on="code")[["label", "count"]]

## Mentions of Countries per Month

As for this tutorial, we are interested in the number of mentions of specific countries and territories over time, we need to decide on a time unit of analysis. Since Parliament usually meets monthly in Strasbourg, we decide to count the mentions per month.
This requires us to convert the timestamp to a proper `datetime` object and afterwards extracting its month:

In [ ]:
reports_with_areas['timestamp'] = pd.to_datetime(reports_with_areas['timestamp'], format='ISO8601')
reports_with_areas['month'] = reports_with_areas['timestamp'].dt.to_period('M')

As we are interested in the number of mentions per month **per geo area**, we will use `pivot_table` to effectively group by geo area, counting the relevant reports per month and storing these values in a separate colum for each geo area:

In [ ]:
monthly_reports = pd.pivot_table(
    reports_with_areas,
    index='month',
    columns='geo_area_code',
    values='timestamp',
    aggfunc='count',
    fill_value=0
).reset_index()
monthly_reports.head(8)

Taking a look at the output above, we can spot that this approach creates gaps in our dataframe for months in which no geo coded votes took place. For example, the March to May 2020 are missing. In the following code will ensure that we have a row for each month (between the earliest and latest month in the data), filling the columns with 0 for months that we now artificially add for the sake of completeness:

In [ ]:
date_range = pd.date_range(start=reports_with_areas['month'].min().to_timestamp(), end=reports_with_areas['month'].max().to_timestamp(), freq='MS')
all_months = pd.DataFrame({'month': pd.to_datetime(date_range).to_period('M')})

monthly_reports = pd.merge(all_months, monthly_reports, on='month', how='left').fillna(0)
monthly_reports = monthly_reports.reset_index(drop=True)
monthly_reports.head(9)

## Analysis and Visualization

With this dataframe, we are now ready to analyze patterns and trend over time. For this tutorial, we will focus on two countries: Russia and Ukraine, plotting how often Parliament voted on matters relating to these countries over time:

In [ ]:
# Subset the data frame
ukr_rus = monthly_reports[["month", "UKR", "RUS"]].copy()
# convert month to date time again for nice plotting
ukr_rus["month"] = ukr_rus["month"].dt.to_timestamp()

In [ ]:
import numpy as np

plt.figure(figsize=(12, 6))

x = np.arange(len(ukr_rus['month']))
width = 0.35
gap = 0.05
# Side-by-side Barchart with small gaps between Bars
plt.bar(x - width/2 - gap/2, ukr_rus['UKR'], width, label="Ukraine")
plt.bar(x + width/2 + gap/2, ukr_rus['RUS'], width, label="Russia")
plt.title('EP roll-call votes over time')

plt.xlabel('Month')
plt.ylabel('Counts')
# Extract Month as word and Year for nicer Axis Ticks
formatted_dates = [date.strftime('%b %Y') for date in ukr_rus['month']]
# Only tick every third month
plt.xticks(x[::3], [formatted_dates[i] for i in range(0, len(formatted_dates), 3)], rotation=45, fontsize=8)
plt.legend()
plt.show()

This quick, eploratory visualization already shows some interesting trends that intuitively make sense: For both Russia and Ukraine, we can see a stark peak and also overall increase in related votes with Russias large scale invasion in Ukraine in February 2022. In general, we can see a lot of similar movements for both countries, which could indicate that many votes relate to both countries at once, for example [resolutions condemn this invasion itself](https://howtheyvote.eu/votes/165965). For most months, we also see more votes related to Ukraine than to Russia.

## Further Analysis

This is of course only a very rudimentary analysis, but provides us with an interesting starting point. Some further investigations could include:

- relationships between other countries
- analyzing mentions per continent
- analyzing mentions per type of text voted on (i.e., legislative or non-legislative)

## Wrapping Up

In this tutorial we have seen how we can leverage the `geo_area_votes` table of the HowTheyVote data set to identify which texts that Parliament voted on relate to which countries. The tables can easily be combined by joining on the `id` of each vote.

## Feedback?

Was this tutorial helpful? Do you have any feedback for us how to further improve it? Do you have requests for specific tutorials we should provide in the future? Let us know by filling out [this (very short) form](https://tally.so/r/nrbogMs)!